In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-274542
Azure region: eastus2
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-274542


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "optmizing-ml"
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
### YOUR CODE HERE ###
compute_config = AmlCompute.provisioning_configuration(
    vm_size="Standard_D2_V2",
    max_nodes=4
)

compute_target = ComputeTarget(workspace=ws,name= cluster_name)

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': uniform(0.1,1),
        '--max_iter':choice(50,100)
    }
)

# Specify a Policy
policy = BanditPolicy(
    evaluation_interval=1,
    slack_factor=0.1
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./training/conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory="./",
    script="train.py",
    compute_target=compute_target,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="accuracy",
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10,
    max_concurrent_runs=4
)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hdr = exp.submit(hyperdrive_config)  # Submete o job para execução
hdr.wait_for_completion(show_output=True)  # Espera a conclusão do job
assert(hdr.get_status() == "Completed")

RunId: HD_fa21cef8-633f-4b31-b323-68d9c00443dd
Web View: https://ml.azure.com/runs/HD_fa21cef8-633f-4b31-b323-68d9c00443dd?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-274542/workspaces/quick-starts-ws-274542&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-03-03T00:31:30.3787050Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-03-03T00:31:30.6874681Z][SCHEDULER][INFO]Scheduling job, id='HD_fa21cef8-633f-4b31-b323-68d9c00443dd_0' 
[2025-03-03T00:31:30.7479018Z][SCHEDULER][INFO]Scheduling job, id='HD_fa21cef8-633f-4b31-b323-68d9c00443dd_1' 
[2025-03-03T00:31:30.7920820Z][SCHEDULER][INFO]Scheduling job, id='HD_fa21cef8-633f-4b31-b323-68d9c00443dd_2' 
[2025-03-03T00:31:30.7929910Z][SCHEDULER][INFO]Scheduling job, id='HD_fa21cef8-633f-4b31-b323-68d9c00443dd_3' 
[2025-03-03T00:31:31.3316117Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_fa21cef8-633f-4b31-b323-68d9c00443dd_3' 
[2025-03-03T00:3

In [5]:
import joblib
# Get your best run and save the model from that run.
best_run = hdr.get_best_run_by_primary_metric()


In [6]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_fa21cef8-633f-4b31-b323-68d9c00443dd_8,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"  # Substitua pela URL real
ds = TabularDatasetFactory.from_delimited_files(path=url)

In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [9]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=2,
    compute_target= compute_target)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config, show_output =True)
remote_run.wait_for_completion()

Submitting remote run.
No run_configuration provided, running on optmizing-ml with default configuration
Running on remote compute: optmizing-ml


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_c76ea55d-2cda-4006-a87d-01fcf97ae184,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = remote_run.get_output()